In [12]:
import os
import sys
sys.path.append('/home/cocoza4/workspace/absorouteio/asr-face-recognition/src')
import math
import time
import argparse
import logging
import numpy as np
import tensorflow as tf
from datetime import datetime
from functools import partial
from pathlib import Path
import tensorflow.keras.backend as K

import utils
import losses
import models
from evaluate import lfw, cfp, eval_utils
from generators import TFRecordDataGenerator
from backbone.resnet import SEResNet 
logging.basicConfig(level=logging.INFO)


@tf.function
def predict_embedding(model, images):
    x = model(images, training=False)
    return tf.nn.l2_normalize(x, axis=-1)

@tf.function
def train_step(model, inputs, labels, emb_weights, optimizer, loss_fn, global_batch_size):
    with tf.GradientTape(persistent=False) as tape:
        embeddings = model(inputs, training=True)
        per_example_loss = loss_fn(embeddings, emb_weights, labels)
        loss = tf.nn.compute_average_loss(per_example_loss, global_batch_size=global_batch_size)

    trainable_vars = model.trainable_variables + [emb_weights]
    gradients = tape.gradient(loss, trainable_vars)
    optimizer.apply_gradients(zip(gradients, trainable_vars))

    return loss

def parse_example(proto):
    feature_description = {
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/label': tf.io.FixedLenFeature([], tf.int64),
        'image/width': tf.io.FixedLenFeature([], tf.int64),
        'image/height': tf.io.FixedLenFeature([], tf.int64)
    }
    tf_example = tf.io.parse_single_example(proto, feature_description)
    return tf_example

def _preprocess(image, training):
    if training:
        image = tf.image.random_flip_left_right(image)
    image = tf.cast(image, tf.float32)
    image -= 127.5
    image *= 0.0078125
    return image

def preprocess_tf_example(example, training=True):
    width = example['image/width']
    height = example['image/height']
    label = tf.cast(example['image/label'], tf.int32)
    image = tf.io.decode_image(example['image/encoded'])
    return _preprocess(image, training=training), label
    
def preprocess(path, training=True):
    raw = tf.io.read_file(path)
    image = tf.image.decode_image(raw)
    _, _, c = image.shape
    if c > 3 or c == 1:
        image = utils.to_rgb(image.numpy())
    return _preprocess(image, training=training)

def load_ckpt(ckpt_dir, max_to_keep, backbone, global_step, model, optimizer, emb_weights):
    ckpt = tf.train.Checkpoint(global_step=global_step, model=model, optimizer=optimizer, emb_weights=emb_weights)
    ckpt_manager = tf.train.CheckpointManager(ckpt, ckpt_dir, max_to_keep=max_to_keep, checkpoint_name=backbone)
 
    if ckpt_manager.latest_checkpoint:
        ckpt.restore(ckpt_manager.latest_checkpoint)
        logging.info("Restored from {}".format(ckpt_manager.latest_checkpoint))
    else:
        logging.info("Initializing from scratch.")

    return ckpt_manager

def evaluate(summary, global_step, eval_fn, predict_fn, tag, results_file):
    t1 = time.time()
    accuracy, val, far, frr = eval_fn(predict_fn=predict_fn)
    time_elapsed = time.time() - t1
    summary.scalar('%s/accuracy' % tag, accuracy, step=global_step)
    summary.scalar('%s/val_rate' % tag, val, step=global_step)
    summary.scalar('%s/far' % tag, far, step=global_step)
    summary.scalar('%s/frr' % tag, frr, step=global_step)
    summary.scalar('%s/time_elapsed' % tag, time_elapsed, step=global_step)
    eval_utils.save_result(results_file, accuracy, val, far, frr)


def get_optimizer(name, lr, **kwargs):
    if name.lower() == 'adam':
        opt = tf.keras.optimizers.Adam(lr)
    elif name.lower() == 'sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=lr, momentum=kwargs['mom'], nesterov=True)
    return opt

In [2]:
n_classes = 19
m1 = 1.
m2 = 0.5
m3 = 0
s = 64.
mom = 0.9
batch_size = 10
embedding_size = 10

train_dir = '/home/cocoza4/datasets/sample_asian_and_msra_mtcnnpy_112_margin32'

In [3]:
def predict_fn(inputs):
    preprocessed = np.array([preprocess(path, training=False).numpy() for path in inputs])
    return predict_embedding(model, preprocessed)


loss_fn = partial(losses.arcface_loss, n_classes=n_classes, m1=m1, m2=m2, m3=m3, s=s, 
                    reduction=tf.keras.losses.Reduction.NONE)

In [4]:
strategy = tf.distribute.MirroredStrategy()

W0415 01:42:53.993669 139684702295872 cross_device_ops.py:1258] There are non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.


In [5]:
log_template = 'Epoch: %d[%d/%d]\tStep %d\tTime %.3f\tLoss %2.3f\tlr %.5f'

train_gen = TFRecordDataGenerator(train_dir, batch_size=batch_size)
train_ds = train_gen.generate(example_parser=parse_example, preprocess_fn=preprocess_tf_example)
    
global_batch_size = 10 * strategy.num_replicas_in_sync
train_iter = iter(train_ds)

global_batch_size

10

In [6]:
epoch = 0
steps_per_epoch = 2

In [7]:
with strategy.scope():
    global_step = tf.Variable(0, name="global_step", dtype=tf.int64, trainable=False)
    current_lr = 0.1
    optimizer = get_optimizer('sgd', current_lr, mom=mom)

    backbone = SEResNet(blocks=[1, 1, 1, 1])
    model = models.ArcFaceModel(backbone, embedding_size)
    
    initializer = tf.initializers.VarianceScaling()
    emb_weights = tf.Variable(initializer(shape=[n_classes, embedding_size]), 
                                name='embedding_weights', dtype=tf.float32)

    for step in range(steps_per_epoch):
        inputs, labels = next(train_iter)
        t1 = time.time()
        per_replica_loss = strategy.experimental_run_v2(train_step, args=(model, inputs, labels, emb_weights, 
                                                                            optimizer, loss_fn, global_batch_size,))
        loss = strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_loss, axis=None)
        elapsed = time.time() - t1

        print(log_template % (epoch+1, step+1, steps_per_epoch, global_step.numpy(), elapsed, loss.numpy(), current_lr))

        global_step.assign_add(1)


Epoch: 1[1/2]	Step 0	Time 6.832	Loss 67.240	lr 0.10000
Epoch: 1[2/2]	Step 1	Time 2.485	Loss 62.972	lr 0.10000


In [17]:
optimizer.lr.numpy()

0.1

In [48]:
optimizer.lr.assign(0.5)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=0.5>

In [29]:
class CyclicLR:
    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None, new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
    
    @property
    def lr(self):
        if self.clr_iterations == 0:
            return self.base_lr
        else:
            return self.clr()
    
    def step(self):
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(self.lr)
        self.history.setdefault('iterations', []).append(self.trn_iterations)
    

In [44]:
clr = CyclicLR(base_lr=0.01, max_lr=0.1, mode='triangular')
clr

In [45]:
clr.lr

0.01

In [46]:
lrs = []
for _ in range(10000):
    lrs.append(clr.lr)
    clr.step()

In [47]:
lrs

[0.01,
 0.010044999999999995,
 0.01008999999999999,
 0.010135000000000005,
 0.01018,
 0.010224999999999996,
 0.010269999999999991,
 0.010315000000000005,
 0.010360000000000001,
 0.010404999999999996,
 0.01044999999999999,
 0.010495000000000006,
 0.01054,
 0.010584999999999995,
 0.01062999999999999,
 0.010675000000000006,
 0.01072,
 0.010764999999999997,
 0.010809999999999991,
 0.010855000000000005,
 0.010900000000000002,
 0.010944999999999996,
 0.010989999999999991,
 0.011035000000000007,
 0.011080000000000001,
 0.011124999999999996,
 0.01116999999999999,
 0.011215000000000006,
 0.011260000000000001,
 0.011304999999999996,
 0.011349999999999992,
 0.011395000000000006,
 0.011440000000000002,
 0.011484999999999997,
 0.011529999999999992,
 0.011575000000000007,
 0.011620000000000002,
 0.011664999999999997,
 0.011709999999999991,
 0.011755000000000007,
 0.011800000000000001,
 0.011844999999999998,
 0.011889999999999993,
 0.011935000000000008,
 0.011980000000000003,
 0.012024999999999997,
 

In [ ]:
class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())